# 基于Wiki中文语料库的语言模型

In [1]:
import jieba, os, re

In [2]:
filepath = 'D:\\wiki\CC\wiki_corpus00'

In [3]:
os.path.exists(filepath)

True

In [4]:
def generate_zhwiki_list(file_path):
    file = open(file_path, "r", encoding="utf-8")
    content_line = file.readline()
    article_contents_list = []
    i = 0
    while content_line:
        temp = ''
        content_line = content_line.strip("\n")
        if len(content_line) > 0:
            for word in content_line:
                if word == ' ':
                    article_contents_list += [temp]
                    temp = ''
                else:
                    temp = temp + word
        content_line = file.readline()
        if temp != '': article_contents_list += [temp]
    return article_contents_list

In [5]:
zh_wiki_list = generate_zhwiki_list(filepath)

In [6]:
zh_wiki_list

['数学',
 '数学',
 '是',
 '利用',
 '符号语言',
 '研究',
 '数量',
 '结构',
 '变化',
 '以及',
 '空间',
 '等',
 '概念',
 '的',
 '一门',
 '学科',
 '从',
 '某种',
 '角度看',
 '属于',
 '形式',
 '科学',
 '的',
 '一种',
 '数学',
 '透过',
 '抽象化',
 '和',
 '逻辑推理',
 '的',
 '使用',
 '由',
 '计数',
 '计算',
 '量度',
 '和',
 '对',
 '物体',
 '形状',
 '及',
 '运动',
 '的',
 '观察',
 '而',
 '产生',
 '数学家',
 '们',
 '拓展',
 '这些',
 '概念',
 '为了',
 '公式化',
 '新',
 '的',
 '猜想',
 '以及',
 '从',
 '选定',
 '的',
 '公理',
 '及',
 '定义',
 '中',
 '建立',
 '起',
 '严谨',
 '推导',
 '出',
 '的',
 '定理',
 '基础',
 '数学',
 '的',
 '知识',
 '与',
 '运用',
 '总是',
 '个人',
 '与',
 '团体',
 '生活',
 '中',
 '不可或缺',
 '的',
 '一环',
 '对',
 '数学',
 '基本概念',
 '的',
 '完善',
 '早',
 '在',
 '古埃及',
 '美索不达米亚',
 '及',
 '古印度',
 '内',
 '的',
 '古代',
 '数学',
 '文本',
 '便',
 '可观',
 '见',
 '而',
 '在',
 '古希腊',
 '那里',
 '有',
 '更为',
 '严谨',
 '的',
 '处理',
 '从',
 '那时',
 '开始',
 '数学',
 '的',
 '发展',
 '便',
 '持续',
 '不断',
 '地',
 '小幅',
 '进展',
 '至',
 '16',
 '世纪',
 '的',
 '文艺复兴',
 '时期',
 '因为',
 '新',
 '的',
 '科学',
 '发现',
 '和',
 '数学',
 '革新',
 '两者',
 '的',
 '交互',
 '致使',
 '数学',
 '的',
 '加速',
 '发展',
 '

In [37]:
from collections import Counter

In [38]:
all_2_grams_words = [''.join(zh_wiki_list[i:i+2]) for i in range(len(zh_wiki_list[:-2]))]

In [39]:
_2_grams_words_sum = len(zh_wiki_list)
_2_grams_words_dict = Counter(zh_wiki_list)

In [40]:
_2_grams_words_sum

17259553

In [41]:
_2_grams_words_dict

Counter({'数学': 3353,
         '是': 184715,
         '利用': 4537,
         '符号语言': 5,
         '研究': 15214,
         '数量': 3173,
         '结构': 4907,
         '变化': 3581,
         '以及': 20126,
         '空间': 3230,
         '等': 52153,
         '概念': 3583,
         '的': 995671,
         '一门': 556,
         '学科': 2017,
         '从': 23578,
         '某种': 610,
         '角度看': 58,
         '属于': 5722,
         '形式': 5166,
         '科学': 4506,
         '一种': 12459,
         '透过': 2798,
         '抽象化': 48,
         '和': 159423,
         '逻辑推理': 30,
         '使用': 22795,
         '由': 36552,
         '计数': 120,
         '计算': 3199,
         '量度': 174,
         '对': 40938,
         '物体': 1998,
         '形状': 1054,
         '及': 39281,
         '运动': 7632,
         '观察': 1827,
         '而': 58039,
         '产生': 8862,
         '数学家': 1125,
         '们': 4642,
         '拓展': 309,
         '这些': 14188,
         '为了': 7865,
         '公式化': 36,
         '新': 15208,
         '猜想': 405,
         '选定': 

In [42]:
frequences = [f for w, f in _2_grams_words_dict.most_common()]

In [43]:
frequences_sum = sum(frequences)

In [44]:
def get_prob(word):
    esp = 1 / frequences_sum
    if word in _2_grams_words_dict:
        return _2_grams_words_dict[word] / frequences_sum
    else:
        return esp

In [45]:
def get_combination_prob(w1, w2):
    if (w1+w2) in zh_wiki_list:return _2_grams_words_dict[w1 + w2] / _2_grams_words_sum
    else:
        return 1 / _2_grams_words_sum

In [46]:
def get_prob_2_gram(w1, w2):
    return get_combination_prob(w1, w2) / get_prob(w1)

In [47]:
def cut(string):return list(jieba.cut(string))

In [48]:
def language_model_2_grams(stences):
    words = cut(stences)
    sentence_probility = 1

    for i, word in enumerate(words):
        if i == 0:
            prob = get_prob(word)
        else:
            previous = words[i-1]
            prob = get_prob_2_gram(previous, word)
        sentence_probility *= prob
    return sentence_probility

In [49]:
language_model_2_grams('小明今天抽到一台苹果手机')

2.0726062379633708e-17

In [50]:
language_model_2_grams('数学好啊')

1.7187460125280496e-11